In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [3]:
!wget  "https://archive.ics.uci.edu/ml/machine-learning-databases/00526/data.zip"
!unzip "/content/data.zip" 

--2022-10-14 14:13:06--  https://archive.ics.uci.edu/ml/machine-learning-databases/00526/data.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116104947 (111M) [application/x-httpd-php]
Saving to: ‘data.zip.1’

data.zip.1          100%[===================>] 110.73M  50.4MB/s    in 2.2s    

2022-10-14 14:13:09 (50.4 MB/s) - ‘data.zip.1’ saved [116104947/116104947]

Archive:  /content/data.zip
replace BitcoinHeistData.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: BitcoinHeistData.csv    


In [4]:
df=pd.read_csv("/content/BitcoinHeistData.csv")

In [5]:
print (df.head())

                              address  year  day  length    weight  count  \
0   111K8kZAEnJg245r2cM6y9zgJGHZtJPy6  2017   11      18  0.008333      1   
1  1123pJv8jzeFQaCV4w644pzQJzVWay2zcA  2016  132      44  0.000244      1   
2  112536im7hy6wtKbpH1qYDWtTyMRAcA2p7  2016  246       0  1.000000      1   
3  1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7  2016  322      72  0.003906      1   
4  1129TSjKtx65E35GiUo4AYVeyo48twbrGX  2016  238     144  0.072848    456   

   looped  neighbors       income            label  
0       0          2  100050000.0  princetonCerber  
1       0          1  100000000.0   princetonLocky  
2       0          2  200000000.0  princetonCerber  
3       0          2   71200000.0  princetonCerber  
4       0          1  200000000.0   princetonLocky  


In [6]:
df.isnull().sum()

address      0
year         0
day          0
length       0
weight       0
count        0
looped       0
neighbors    0
income       0
label        0
dtype: int64

In [7]:
temp_data = df['address']
temp_label = df['label']

In [8]:
df = df.drop(['address','label'], axis=1)

In [9]:
df.head()

,year,day,length,weight,count,looped,neighbors,income
0,2017,11,18,0.008333,1,0,2,100050000.0
1,2016,132,44,0.000244,1,0,1,100000000.0
2,2016,246,0,1.000000,1,0,2,200000000.0
3,2016,322,72,0.003906,1,0,2,71200000.0
4,2016,238,144,0.072848,456,0,1,200000000.0


In [10]:
for column in df.columns:
    df[column].fillna(df[column].mode()[0], inplace=True)
    print(df[column].mode()[0])

2016
78
0
1.0
1
0
2
100000000.0


In [11]:
tr_size = 0.70
v_size=0.15
t_size = 0.15
df['label'] = temp_label
tr_idx = int(len(temp_label)*tr_size)
df_tr = df[0:tr_idx]
v_idx = int(len(temp_label)*v_size)
df_v = df[tr_idx:tr_idx+v_idx]
df_t = df[tr_idx+v_idx:]
t_idx = int(len(temp_label)*t_size)
X_train, y_train = df_tr.drop(columns='label'), df_tr['label']
X_valid, y_valid = df_v.drop(columns='label'), df_v['label']
X_test, y_test = df_t.drop(columns='label'), df_t['label']   
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(2041687, 8)
(2041687,)
(437504, 8)
(437504,)
(437506, 8)
(437506,)


(None, None)

In [12]:
df = df.drop(['label'],axis =1)

In [13]:
def gini(X_train, X_test, y_train,i):
  
    clf_gini = DecisionTreeClassifier(criterion = "gini",
            random_state = 123,max_depth=i, min_samples_leaf=5)
    clf_gini.fit(X_train, y_train)
    return clf_gini

In [14]:
def entropy(X_train, X_test, y_train,i):
    clf_entropy = DecisionTreeClassifier(
            criterion = "entropy", random_state = 123,
            max_depth = i, min_samples_leaf = 5)
    clf_entropy.fit(X_train, y_train)
    return clf_entropy

In [18]:
train_x, valid_x, test_x = np.split(df.sample(frac=1), [int(.7*len(df)),int(.85*len(df))])
train_y, valid_y, test_y = np.split(temp_label.sample(frac=1), [int(.7*len(temp_label)),int(.85*len(temp_label))])

In [19]:
for i in {4,8,10,15,20}:
  print("")
  print("---------------MAX DEPTH =",i,"---------------")
  clf_entropy = entropy(train_x, test_x, train_y,i)
  print("------------ENTROPY-----------------")
  print("")
  yentropypred = clf_entropy.predict(test_x)
  print("ACCURACY SCORE: ")
  print(accuracy_score(test_y,yentropypred)*100)
  clf_gini = gini(train_x, test_x, train_y,i)
  print("-------------------------------------------------------------")
  print("-----------GINI INDEX----------------")
  yginipred = clf_gini.predict(test_x)
  print("ACCURACY SCORE: ")
  print(accuracy_score(test_y,yginipred)*100)


---------------MAX DEPTH = 4 ---------------
------------ENTROPY-----------------

ACCURACY SCORE: 
98.57167346658895
-------------------------------------------------------------
-----------GINI INDEX----------------
ACCURACY SCORE: 
98.57167346658895

---------------MAX DEPTH = 8 ---------------
------------ENTROPY-----------------

ACCURACY SCORE: 
98.57167346658895
-------------------------------------------------------------
-----------GINI INDEX----------------
ACCURACY SCORE: 
98.57167346658895

---------------MAX DEPTH = 10 ---------------
------------ENTROPY-----------------

ACCURACY SCORE: 
98.57121632895624
-------------------------------------------------------------
-----------GINI INDEX----------------
ACCURACY SCORE: 
98.57167346658895

---------------MAX DEPTH = 15 ---------------
------------ENTROPY-----------------

ACCURACY SCORE: 
98.55933075050571
-------------------------------------------------------------
-----------GINI INDEX----------------
ACCURACY SCORE: 


In [17]:
#2 
print("^^^^^^^^^^^^^^^^^^RandomForest^^^^^^^^^^^^^^^^^")
clf = BaggingClassifier(n_estimators=100, max_samples=0.5 ,random_state=123,base_estimator = DecisionTreeClassifier(criterion='gini',max_depth=3))
clf.fit(train_x,train_y)
print("ACCURACY",clf.score(test_x,test_y))
print("---------------------------------------------")

^^^^^^^^^^^^^^^^^^RandomForest^^^^^^^^^^^^^^^^^
ACCURACY 0.9858744471491755
---------------------------------------------


In [ ]:
#3
print("^^^^^^^^^^^^^^^^^^AdaBoostClassifier^^^^^^^^^^^^^^^^^")
for i in {4,8,10,15,20}:
  clf = AdaBoostClassifier(n_estimators=i,base_estimator = DecisionTreeClassifier(criterion='gini',max_depth=15))


  model = clf.fit(train_x, train_y)
  print("------------N-ESTIMATORS: ",i,"--------------")
  print(clf.score(test_x,test_y)*100)
  print("---------------------------------------------")

^^^^^^^^^^^^^^^^^^AdaBoostClassifier^^^^^^^^^^^^^^^^^
------------N-ESTIMATORS:  4 --------------
97.6443697786311
---------------------------------------------
------------N-ESTIMATORS:  8 --------------
95.36348156021074
---------------------------------------------
------------N-ESTIMATORS:  10 --------------
95.02245688620702
---------------------------------------------
------------N-ESTIMATORS:  15 --------------
95.71730608793042
---------------------------------------------
------------N-ESTIMATORS:  20 --------------
97.69648346876035
---------------------------------------------
